In [4]:
pip install pydub
apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [3]:
pip install sarvamai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00


Convert file format to .wav file

In [21]:
from pydub import AudioSegment
import os

def convert_audio_to_wav(input_path, output_path=None):
    """Convert any supported audio file to WAV format."""
    input_ext = os.path.splitext(input_path)[1][1:].lower()

    output_path = output_path or os.path.splitext(input_path)[0] + ".wav"

    try:
        audio = AudioSegment.from_file(input_path, format=input_ext)
    except Exception as e:
        raise RuntimeError(f"Failed to read input audio. Make sure ffmpeg is installed and the format is supported. Details: {e}")

    try:
        audio.export(output_path, format="wav")
    except Exception as e:
        raise RuntimeError(f"Failed to export WAV file. Details: {e}")

    return output_path

def main():
    input_path = input("Enter the path to the audio file: ").strip()
    if not os.path.isfile(input_path):
        print("File not found. Please check the path.")
        return

    output_name = input("Enter output WAV filename (or press Enter to use default): ").strip()
    output_path = output_name if output_name else None

    try:
        result = convert_audio_to_wav(input_path, output_path)
        print(f"Audio successfully converted to WAV: {result}")
    except Exception as e:
        print(f"Conversion failed: {e}")

if __name__ == "__main__":
    main()


Enter the path to the audio file: /content/audio1.ogg
Enter output WAV filename (or press Enter to use default): input1.wav
Audio successfully converted to WAV: input1.wav


Speech to text translate

In [ ]:
import requests

API_KEY = "enter_api_key"

def translate_speech():
    audio_path = input("Enter the path to the audio file: ").strip()
    source_lang = "unknown"
    target_lang = "en-IN"

    url = "https://api.sarvam.ai/speech-to-text-translate"
    headers = {
        "api-subscription-key": API_KEY,
        "accept": "application/json"
    }

    try:
        with open(audio_path, "rb") as audio_file:
            files = {
                "file": (audio_path, audio_file, "audio/wav")
            }
            data = {
                "task": "translate",
                "language_code": source_lang,
                "target_language_code": target_lang,
                "model": "saaras:v2"
            }

            response = requests.post(url, headers=headers, files=files, data=data)

            if response.status_code == 200:
                result = response.json()
                print("\nSpeech Translation Result")
                print(f"Source Language Detected: {result.get('language_code', 'Unknown')}")
                print("Transcription (in source or intermediate language):")
                print(f"> {result.get('transcript', 'No transcript available')}")
                print(f"\nRequest ID: {result.get('request_id', 'N/A')}")
                diarized = result.get('diarized_transcript')
                print(f"Speaker Separation (Diarization): {'Available' if diarized else 'None'}\n")
            else:
                print(f"Error {response.status_code}:")
                print(response.text)

    except FileNotFoundError:
        print("File not found. Please check the path and try again.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

translate_speech()

Enter the path to the audio file: /content/output.wav

Speech Translation Result
Source Language Detected: ta-IN
Transcription (in source or intermediate language):
> Hello, I am a shopping assistant who helps you make purchases easily. If you tell me what products you are looking for, I will immediately provide complete information about them, price comparisons, and recommendations.

Request ID: 20250521_96bf60fb-84aa-4eee-8312-1ae5b3ea2ae5
Speaker Separation (Diarization): None



Text to Speech

In [ ]:
import requests
from sarvamai import SarvamAI
from sarvamai.play import save

API_KEY = "enter_api_key"

def detect_language(text):
    """Detect the language and script of the input text."""
    url = "https://api.sarvam.ai/text-lid"
    headers = {
        "api-subscription-key": API_KEY,
        "Content-Type": "application/json"
    }
    payload = { "input": text }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result.get("language_code", "und"), result.get("script_code", "unknown")
    else:
        raise Exception(f"Language detection failed: {response.text}")

def convert_text_to_speech(text, language_code, output_file="output.wav", speaker="manisha", model="bulbul:v2"):
    """Convert the given text to speech and save it as an audio file."""
    client = SarvamAI(api_subscription_key=API_KEY)
    audio = client.text_to_speech.convert(
        target_language_code=language_code,
        text=text,
        model=model,
        speaker=speaker
    )
    save(audio, output_file)
    print(f"Audio saved as '{output_file}'")

def main():
    text = input("Enter the text you want to convert to speech: ").strip()
    if not text:
        print("No text entered. Exiting.")
        return

    try:
        language_code, script_code = detect_language(text)
        print(f"Language Detected: {language_code} | Script: {script_code}")
        convert_text_to_speech(text, language_code)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Enter the text you want to convert to speech: வணக்கம்! நான் உங்கள் எளிதாகக் கொள்முதல் செய்ய உதவும் ஷாப்பிங் உதவியாளர். நீங்கள் தேடும் பொருள்கள் எவை என்பதை எனக்குச் சொன்னால், அவற்றைப் பற்றிய முழுமையான தகவல்களையும், விலையிலான ஒப்பீடுகளையும், பரிந்துரைகளையும் உடனடியாக வழங்குகிறேன். உதாரணமாக, நீங்கள் ஒரு புதிய மொபைல் போன் வாங்க விரும்பினால், உங்கள் தேவைகளுக்கு ஏற்ப சிறந்த விருப்பங்களைத் தேர்ந்தெடுத்து உங்களுக்குத் தருகிறேன். மேலும், தள்ளுபடி விவரங்கள், வாடிக்கையாளர் மதிப்பீடுகள், மற்றும் பொருள் கிடைப்பதைப் பற்றிய விவரங்களும் உங்களுக்காகத் தேடி தருகிறேன். இப்போது நீங்கள் நேரத்தைச் சேமித்து, புத்திசாலித்தனமாக வாங்க முடியும். எனது உதவியுடன் உங்கள் ஷாப்பிங் அனுபவம் எளிமையாகவும் சிரமமில்லாததாகவும் இருக்கும்!
Language Detected: ta-IN | Script: Taml
Audio saved as 'output.wav'
